<a href="https://colab.research.google.com/github/SlayerDraco/pr0mp7_3nh4nc3r/blob/main/pr0mp7_3nh4nc3r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# New Cell: Installation and Setup
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata

# Configure the API client with your secret key
try:
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    # Initialize the model
    llm = genai.GenerativeModel('models/gemini-flash-latest')
    print("Gemini model configured successfully.")
except Exception as e:
    llm = None
    print(f"Error configuring Gemini model: {e}")

In [ ]:
%%capture
!pip install symspellpy
!pip install textblob
!pip install scikit-learn pandas joblib
!python -m textblob.download_corpora

In [ ]:
from symspellpy.symspellpy import SymSpell, Verbosity
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from datetime import datetime
import os
import joblib
import random
import pandas as pd
import csv


In [ ]:
# --- ML Model Training Cell ---
# This cell trains the model only if the model file doesn't exist.

model_filename = 'intent_model.joblib'
data_filename = 'training_dataset.csv'

if not os.path.exists(model_filename):
    print(f"[*] Model file '{model_filename}' not found. Training a new model...")
    try:
        # 1. Load Data
        df = pd.read_csv(data_filename)

        # 2. Define Features (X) and Target (y)
        X = df['prompt']
        y = df['intent']

        # 3. Create and Train the ML Pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
            ('clf', LogisticRegression(random_state=42)),
        ])
        pipeline.fit(X, y)

        # 4. Save the trained pipeline
        joblib.dump(pipeline, model_filename)
        print(f"[+] Model trained and saved to '{model_filename}'")

    except FileNotFoundError:
        print(f"[!] Error: Training data '{data_filename}' not found. Please create or upload it.")
        intent_model = None
else:
    print(f"[*] Model file '{model_filename}' found. Skipping training.")

# --- Load the Model ---
try:
    intent_model = joblib.load(model_filename)
    print("[+] Intent model loaded successfully.")
except FileNotFoundError:
    print("[!] Failed to load the model.")
    intent_model = None

In [ ]:
dictionary_path = "frequency_dictionary_en_82_765.txt"
max_edit_distance_dictionary = 2
prefix_length = 7

sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    print(f"[!] Failed to load dictionary from {dictionary_path}")

# Custom technical/AI words to preserve
custom_words = [
    "python", "javascript", "html", "react", "robot",
    "dataset", "API", "GPT", "AI", "ML", "cybersecurity",
    "blockchain", "deepfake", "docker", "kubernetes", "quiksort",
    "matlab", "waf", "sql", "geolocation", "osint", "payload",
    "BCC", "FCC"
]

In [ ]:
def correct_with_symspell(text: str) -> str:
    corrected_words = []
    for word in text.split():
        # Preserve custom words (technical, slang, names)
        if word.lower() in [w.lower() for w in custom_words]:
            corrected_words.append(word)
            continue

        # Skip very short words (1-2 letters), likely abbreviations/slang
        if len(word) <= 2:
            corrected_words.append(word)
            continue

        # Get SymSpell suggestions
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)

        if suggestions:
            # Only accept suggestion if it's a real improvement (edit distance > 0)
            best_suggestion = suggestions[0].term
            if best_suggestion.lower() != word.lower():
                corrected_words.append(best_suggestion)
            else:
                corrected_words.append(word)
        else:
            # No suggestions, keep original word
            corrected_words.append(word)

    return " ".join(corrected_words)

In [ ]:
def get_intent_hybrid(text: str) -> str:
    """
    Detects intent using a hybrid approach:
    1. Get a prediction from the ML model.
    2. Use rule-based keywords to override and correct the prediction if necessary.
    """
    # Step 1: Get the baseline prediction from the ML model
    if not intent_model:
        return "GENERAL_QUERY" # Fallback

    predicted_intent = intent_model.predict([text])[0]

    text_lower = text.lower()
    # Rule for PROJECT_BRIEF
    project_keywords = ['build an interactive', 'design a tool', 'create a concept', 'design a dashboard', 'plan a database']
    if any(keyword in text_lower for keyword in project_keywords):
        return "PROJECT_BRIEF"

    # Rule for TECHNICAL_EXPLANATION
    explanation_keywords = ['how to bypass', 'explain the concept', 'what is the difference','function to solve']
    if any(keyword in text_lower for keyword in explanation_keywords):
        return "TECHNICAL_EXPLANATION"

    # Step 3: If no specific rules are met, trust the original ML prediction.
    return predicted_intent


In [ ]:
# Updated Function: Replaces the previous enhance_with_llm

def enhance_with_llm(intent: str, prompt: str) -> str:
    """
    Uses an LLM (Gemini) to perform the final, knowledge-driven enhancement,
    with constraints on output length.
    """
    if not llm:
        return "Error: LLM not configured. Cannot perform enhancement."

    # A dictionary of expert personas and instructions for the LLM.
    # Now updated with length constraints.
    meta_prompts = {
        "PROJECT_BRIEF": """
            You are an expert project manager. A user will provide a vague project idea.
            Rewrite it into a detailed but concise project specification, NOT TO EXCEED 8 LINES.
            The specification must define:
            1. The core product and its target audience.
            2. 2-3 essential key features or user stories.
            3. The desired tone or style (e.g., professional, modern, minimalist).
        """,
        "TECHNICAL_EXPLANATION": """
            You are an expert technical writer. A user will ask a vague technical question.
            Rewrite it into a detailed but concise prompt for an expert system, NOT TO EXCEED 8 LINES.
            The prompt must clearly state:
            1. The core topic.
            2. 3-4 specific sub-topics or concepts that must be covered in the answer.
            3. The intended audience (e.g., beginner, university student, professional).
        """,
        "IMAGE_CREATION": """
            You are a professional art director. A user will describe an image.
            Rewrite it into a hyper-detailed but concise prompt for an image generation AI, NOT TO EXCEED 8 LINES.
            The prompt must specify:
            1. The primary subject and composition.
            2. The artistic style (e.g., photorealism, digital painting, anime).
            3. Specific details about lighting, camera angle, and color palette.
        """,
        "CODE_GENERATION": """
            You are an expert software architect. A user will ask for a script.
            Rewrite their request into a clear and concise code specification for a senior developer, NOT TO EXCEED 8 LINES.
            The prompt must specify:
            1. The core function of the script.
            2. The required inputs and expected outputs.
            3. Key requirements like error handling, comments, or specific libraries to use.
        """,
        "CREATIVE_WRITING": """
            You are a seasoned story editor. A user will provide a story idea.
            Rewrite it into a compelling story brief for a professional author, NOT TO EXCEED 8 LINES.
            The brief must establish:
            1. The genre and setting (e.g., cyberpunk, fantasy).
            2. The protagonist and their core conflict or goal.
            3. The central theme and desired tone of the story.
        """
    }

    instructions = meta_prompts.get(intent, "You are a helpful assistant. Rewrite the following user prompt to be more clear and detailed, keeping it under 8 lines.")

    final_prompt_for_llm = f"{instructions}\n\nUser's Prompt: '{prompt}'"

    try:
        response = llm.generate_content(final_prompt_for_llm)
        return response.text.strip()
    except Exception as e:
        return f"Error during LLM enhancement: {e}"

In [ ]:
# Update the main pipeline function

def enhance_prompt(broken_prompt: str, show_corrections: bool = True) -> str:
    """
    The complete, final pipeline.
    """
    print(f"\n--- Enhancing Prompt ---")

    corrected = correct_with_symspell(broken_prompt)
    intent = get_intent_hybrid(corrected)

    # --- HERE IS THE SWAP ---
    # Replace the old keyword expander with the new LLM enhancer
    final_enhanced_prompt = enhance_with_llm(intent, corrected)

    if show_corrections:
        print(f"[Original]   {broken_prompt}")
        print(f"[Corrected]  {corrected}")
        print(f"[Intent]     {intent} (Predicted by Hybrid Model)")
        print("\n--- Final LLM-Enhanced Prompt ---")

    return final_enhanced_prompt

In [ ]:
def log_feedback(prompt, corrected_prompt, intent, enhanced_prompt, was_correct, corrected_intent=None):
    """
    Logs the enhancer's performance and user feedback to a CSV file.
    """
    log_file = 'feedback_log.csv'
    file_exists = os.path.exists(log_file)

    fieldnames = [
        'timestamp', 'original_prompt', 'corrected_prompt',
        'predicted_intent', 'enhanced_prompt', 'was_correct', 'user_corrected_intent'
    ]

    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()  # Write headers only if the file is new

        writer.writerow({
            'timestamp': datetime.now().isoformat(),
            'original_prompt': prompt,
            'corrected_prompt': corrected_prompt,
            'predicted_intent': intent,
            'enhanced_prompt': enhanced_prompt,
            'was_correct': was_correct,
            'user_corrected_intent': corrected_intent if corrected_intent else 'N/A'
        })

In [ ]:
# New Final Cell: Main Interactive Loop

print("--- Prompt Enhancer v3.0 ---")
print("Type your prompt below, or type 'quit/q' to exit.")

# We need to get the intent out of the enhance_prompt function for logging,
# so we'll modify it slightly to return both the prompt and the intent.
def enhance_prompt_with_intent(broken_prompt: str):
    corrected = correct_with_symspell(broken_prompt)
    intent = get_intent_hybrid(corrected)
    final_enhanced_prompt = enhance_with_llm(intent, corrected)
    return final_enhanced_prompt, intent, corrected

# This is the main loop
while True:
    print("-" * 30)
    user_prompt = input("Your Prompt: ")

    if user_prompt.lower() in ['quit', 'exit', 'q']:
        print("Exiting session. Goodbye!")
        break

    # Get the enhanced prompt and the data for logging
    enhanced_prompt, predicted_intent, corrected_prompt = enhance_prompt_with_intent(user_prompt)

    print("\n--- LLM-Enhanced Prompt ---")
    print(enhanced_prompt)
    print("-" * 30)

    # --- The Feedback Loop ---
    feedback = ''
    while feedback not in ['y', 'n']:
        feedback = input("Was this enhancement good? (y/n): ").lower()

    if feedback == 'y':
        # "Appreciate": Log the success
        log_feedback(user_prompt, corrected_prompt, predicted_intent, enhanced_prompt, was_correct=True)
        print("[+] Success logged. Thank you!")
    else:
        # "Punish": Log the failure and ask for the correct intent
        print("Sorry about that. To help the model learn, please provide the correct intent.")
        print("Options: [CODE_GENERATION, IMAGE_CREATION, PROJECT_BRIEF, TECHNICAL_EXPLANATION, CREATIVE_WRITING, GENERAL_QUERY]")
        correct_intent = input("What should the intent have been? ")
        log_feedback(user_prompt, corrected_prompt, predicted_intent, enhanced_prompt, was_correct=False, corrected_intent=correct_intent)
        print("[!] Failure and correction logged. This will be used for future training.")